# CBC: Padding

Create a Python program that will be able to encrypt and decrypt the contents of any given file using AES-CBC with key sizes of 128, 192 or 256 bits.

The key and IV to be used should be given as inputs through separate files. In case of an invalid format or invalid length for the key or IV, an appropriate message should be printed to the user.

For this exercise you should use PKCS7 padding: Cryptography - Symmetric Padding

Your program should support the following arguments:

 -e plaintext file: Encrypt file contents
 -d ciphertext file: Decrypt file contents
 -k key file: File containing the key
 -iv IV file: File containing the IV
 -o output file: Output file, if both encryption and decryption are specified, this file should contain both outputs

In [ ]:
import argparse
import sys
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

In [ ]:
def encrypt(plain: bytes, key: bytes, iv: bytes):
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plain)+padder.finalize()

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    return encryptor.update(padded_data) + encryptor.finalize()


def decrypt(ciphertext: bytes, key: bytes, iv: bytes):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()
    return unpadder.update(padded_data)+unpadder.finalize()


In [ ]:

def main(arguments):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        "-e", "--encrypt", help="File to be encrypted", type=str)
    parser.add_argument(
        "-d", "--decrypt", help="File to be decrypted", type=str)
    parser.add_argument(
        "-k", "--key", help="File containing the key", type=str)
    parser.add_argument("--iv", help="File containing the IV",
                        type=str)
    parser.add_argument(
        "-o", "--output", help="The output file", type=argparse.FileType('w'))

    args = parser.parse_args(arguments)

    if not args.key:
        print("Key file must be provided", file=sys.stderr)
        exit(1)
    else:
        key = bytes.fromhex(args.key)

        if len(key) not in (16, 24, 32):
            print("Unacceptable key length. It must be one of 128, 192, 256 bits")
            exit(1)

    if not args.iv:
        print("IV file must be provided", file=sys.stderr)
        exit(1)
    else:
        iv = bytes.fromhex(args.iv)
        if len(iv) != 16:
            print("Unacceptable iv length. It must be 128 bits")
            exit(1)

    if args.encrypt:
        plaintext: bytes = args.encrypt.encode()

        ciphertext = encrypt(plaintext, key, iv)
        if args.output:
            args.output.write(f"Encrypted:\n{ciphertext.hex()}\n")
        else:
            print(f"Encrypted:\n{ciphertext.hex()}")

    if args.decrypt:
        ciphertext: bytes = bytes.fromhex(args.decrypt)

        plaintext = decrypt(ciphertext, key, iv)
        if args.output:
            args.output.write(f"Decrypted:\n{plaintext.decode()}\n")
        else:
            print(f"Decrypted:\n{plaintext.decode()}")




In [ ]:

sys.argv = ["", "-e",  "AES is based on a design principle known as a substitution-permutation network, and is efficient in both software and hardware. Unlike its predecessor DES, AES does not use a Feistel network. AES is a variant of Rijndael, with a fixed block size of 128 bits, and a key size of 128, 192, or 256 bits. By contrast, Rijndael per se is specified with block and key sizes that may be any multiple of 32 bits, with a minimum of 128 and a maximum of 256 bits. Most AES calculations are done in a particular finite field.",
            "-d", "3f9f9ca5114ad204447e6217512803775f992bff0c4751524842e8b40c0eae2b677994ac291d3baaf76665eff2e95ae5361ea168113c46f7f685ae2f716dec51646005d73e033ef2bf4edadfe231b44c95963c9704d8859ceec60a3ca6d0fbe10e4f52d0889c98ab6c95ee91f027af87f5eede40ce291642ee5210be6b8e50940a03f0916143cb2cfec3a95678f8eb3c362f39fd8173c203433f8d2416b9865f3960045b1737dee204d8d98d222c802fb5d932a469f80a96bfd180c8c147a8af73c6b9103ad4c54b262770f6804524eaa0ed4ce50140dfc6c36ac727a5cc3805115173013c8cb0c5dfcc20906b1c0138a037331ac5c233fba713ed52d3eb0910f8d4344e2ed798dfe3121e162492e6cfa83eda22396b81fe9ba2839333bee6b0f6c5208ab6405e3cba3663de227368ec84ebd997473e98aeca7a69d96340be170ed50e7e6868cfe44aa99c62e3bca856543fb4df6a0646e980bd81f860e6fecf9574a3449d19a0df2b6febbfeb3c9ac6973c7c8792516c11d21b0b9b12b22e2178ea4022803d0c65a1e9f887d66eb7d9cb9a7abf9c13267ab902da4f05935270b1a628687eea38ecbfe6a503418ada1fddc411108b0186bdd64e0adb5dc25069814c9bd37a9e90b2634e60077458ca7002846e8d2217dac2c8699fdcc451a2aa6d17217213759ee0f52b52d856105421be06873c4581e653b2fe288020cd55e5574c2e3dc2f37f41a3d4f4e3b1a7f10a",
            "-k", "ecb902004aa569070189aabcacf34effecbb02004aa569073989a00cacf34eff",
            "--iv", "9a090a4fa5692015689aab07acf34e2c",
            "-o", "InputFiles/Q4/dec.txt"
        ]

main(sys.argv[1:])

In [2]:
print("----------------------------------------Plaintext--------------------------------------")
with open("InputFiles/Q4/plaintext.txt", "r") as file:
    content = file.read()
    print(content)
print("----------------------------------------Ciphertext--------------------------------------")
with open("InputFiles/Q4/ciphertext.txt", "r") as file:
    content = file.read()
    print(content)

print("----------------------------------------Key--------------------------------------")
with open("InputFiles/Q4/key.txt", "r") as file:
    content = file.read()
    print(content)
print("----------------------------------------IV--------------------------------------")
with open("InputFiles/Q4/iv.txt", "r") as file:
    content = file.read()
    print(content)
print("----------------------------------------OutPut FIles--------------------------------------")
with open("InputFiles/Q4/dec.txt", "r") as file:
    content = file.read()
    print(content)

----------------------------------------Plaintext--------------------------------------
AES is based on a design principle known as a substitution-permutation network, and is efficient in both software and hardware. 
Unlike its predecessor DES, AES does not use a Feistel network. 
AES is a variant of Rijndael, with a fixed block size of 128 bits, and a key size of 128, 192, or 256 bits. 
By contrast, Rijndael per se is specified with block and key sizes that may be any multiple of 32 bits, with a minimum of 128 and a maximum of 256 bits. 
Most AES calculations are done in a particular finite field.
----------------------------------------Ciphertext--------------------------------------
3f9f9ca5114ad204447e6217512803775f992bff0c4751524842e8b40c0eae2b677994ac291d3baaf76665eff2e95ae5361ea168113c46f7f685ae2f716dec51646005d73e033ef2bf4edadfe231b44c95963c9704d8859ceec60a3ca6d0fbe10e4f52d0889c98ab6c95ee91f027af87f5eede40ce291642ee5210be6b8e50940a03f0916143cb2cfec3a95678f8eb3c362f39fd8173c2034